In [1]:
import numpy as np
import pandas as pd
import random
import os
from simpletransformers.classification import MultiLabelClassificationModel
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import heapq
import math

In [2]:
devD_df = pd.read_csv('dev/devD.tsv', delimiter='\t', header=None)
trainD_df = pd.read_csv('train/trainD.tsv', delimiter='\t', header=None)

# applying groupby() function to 
# group the data on team value. 
tD_gk = trainD_df.groupby([0]) 
  
# Let's print the first entries 
# in all the groups formed. 
tD_gk.first()

trainD_df = tD_gk.aggregate(lambda x: tuple(x))

mlb = MultiLabelBinarizer()

trainD_enc = pd.DataFrame(mlb.fit_transform(trainD_df[1]),
                   columns= mlb.classes_, 
                   index= trainD_df.index).max(level=0)

trainD_enc['file'] = ""
for i in range(trainD_enc.shape[0]):
    f = open("train/text_files_en/"+ trainD_enc.index[i] + ".txt", "r")
    trainD_enc['file'][i] = f.read()
    
# applying groupby() function to 
# group the data on team value. 
tD_gk = devD_df.groupby([0]) 
  
# Let's print the first entries 
# in all the groups formed. 
tD_gk.first()

devD_df = tD_gk.aggregate(lambda x: tuple(x))

mlb = MultiLabelBinarizer()
devD_enc = pd.DataFrame(mlb.fit_transform(devD_df[1]),
                   columns= mlb.classes_, 
                   index= devD_df.index).max(level=0)

devD_enc['file'] = ""
for i in range(devD_enc.shape[0]):
    f = open("dev/text_files_en/"+ devD_enc.index[i] + ".txt", "r")
    devD_enc['file'][i] = f.read()
    
columns_train = trainD_enc.columns
columns_dev = devD_enc.columns

count = 0
for x in columns_dev:
    if x not in columns_train:
        trainD_enc[x] = 0
        
count = 0
for x in columns_train:
    if x not in columns_dev:
        devD_enc[x] = 0 
        
train = trainD_enc['file']
train = pd.DataFrame(train)

del trainD_enc['file']

train['tar'] = trainD_enc.values.tolist()

dev = devD_enc['file']
dev = pd.DataFrame(dev)

del devD_enc['file']

dev['tar'] = devD_enc.values.tolist()

train = pd.concat([train, dev.iloc[:250]])

#dev = dev.iloc[200:]

train = train.reset_index()
#dev = dev.reset_index()

del train[0]
#del dev[0]

train = train.rename(columns={"file": "text", "tar": "labels"})
dev = dev.rename(columns={"file": "text", "tar": "labels", 0:'index'})

/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rishi/anaconda3/envs/tensorflow1.1/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
with open('labelsD.txt', 'w') as f:
    for i in trainD_enc.columns:
        f.write("%s\n" % i)

In [4]:
test = pd.DataFrame()
files = os.listdir('test/text_files_en/')
test['files'] = files
test['text'] = ""
for i in range(len(files)):
    f = open('test/text_files_en/'+ files[i], "r")
    test['text'][i] = f.read()    
test.index = test['files']
del test['files']
test

,text
files,
S1139-76322010000100010-1.txt,A 12-year-old boy attended the Primary Care Pe...
S1130-01082008001100010-1.txt,A 57-year-old woman diagnosed at 44 years of a...
S0034-98872014000100018-1.txt,A 53-year-old woman with a history of type 2 d...
S0211-57352014000400010-1.txt,A 60-year-old patient with no relevant psychia...
S0210-48062009001000016-2.txt,A 75-year-old male presented with a history of...
...,...
S1137-66272011000300021-1.txt,A 77-year-old male patient was admitted to the...
S0210-48062005000300008-1.txt,"A 45-year-old patient, with no personal histor..."
S1130-01082008000900008-1.txt,A 28-year-old Ecuadorian woman living in Spain...


In [5]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenizer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

stemmer = PorterStemmer()

def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

In [6]:
train['text'] = train['text'].apply(lambda x: remove_punctuation(x), 1)
train['text'] = train['text'].apply(lambda x: tokenizer.tokenize(x.lower()))
train['text'] = train['text'].apply(lambda x: remove_stopwords(x))
train['text'] = train['text'].apply(lambda x: word_lemmatizer(x))
#train['text'] = train['text'].apply(lambda x: word_stemmer(x))

In [7]:
def split_sentences(df, leng, overlap):               # leng is excluding the overlap
    new_df = pd.DataFrame(columns=df.columns)         # new df with same coulmns
    row = 0
    for i in range(df.shape[0]):                      # loop through df rows
        words = df['text'].loc[i].split()
        count = 0
        for j in range(math.ceil(len(words)/leng)):   #loop through sentence
            new_df.loc[row] = df.loc[i]
            new_df['text'].loc[row] = ""
            for k in range (leng + overlap):
                if(j*leng+k < len(words)):
                    new_df['text'].loc[row] += words[j*leng + k] + " "
            row += 1
    return new_df

In [8]:
temp = split_sentences(train, 80, 20)
temp

,text,labels
0,describe case 37yearold man previous active li...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,present head neck auscultation cardiac arrhyth...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,glucose 117 mgdl urea 29 mgdl creatinine 09 mg...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,week admission discharged home completed antib...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,report case 29yearold woman underwent pelvic u...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
2141,muscle weakness analytical data included mild ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2142,patient recovered bone pain muscle weakness pa...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2143,73yearold woman history hypertensive heart dis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2144,sato289 analytical analysis hemogram biochemis...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
electra = MultiLabelClassificationModel(
    "electra", 
    "emilyalsentzer/Bio_ClinicalBERT", 
    num_labels= 2194,
    args={"num_train_epochs": 100, "train_batch_size": 20, "fp16_opt_level" : 'O0',
     "overwrite_output_dir": True,  "learning_rate": 4e-5,
          "max_seq_length": 128, "save_eval_checkpoints": False, "save_model_every_epoch": False,
         "output_dir": "outputs/",
    "cache_dir": "cache/",
    "best_model_dir": "outputs/best_model/"},
)

electra.train_model(temp)

In [10]:
predictions, raw_outputs = electra.predict(test['text'])
test['labels'] = predictions
test['probabilities'] = raw_outputs.tolist()
test.to_csv('electraD.csv', header = True, index = True)

Converting to features started. Cache is not used.


In [4]:
xlnet = MultiLabelClassificationModel(
    "xlnet", 
    "xlnet-base-cased", 
    num_labels= 2194,
    args={"num_train_epochs": 100, "train_batch_size": 20, "fp16_opt_level" : 'O0',
     "overwrite_output_dir": True,  "learning_rate": 4e-5,
          "max_seq_length": 128, "save_eval_checkpoints": False, "save_model_every_epoch": False,
         "output_dir": "outputs/",
    "cache_dir": "cache/"}
)

xlnet.train_model(temp)

In [ ]:
predictions, raw_outputs = xlnet.predict(test['text'])
del test['labels']
del test['probabilities']
test['labels'] = predictions
test['probabilities'] = raw_outputs.tolist()
test.to_csv('xlnetD.csv', header = True, index = True)

In [12]:
roberta = MultiLabelClassificationModel(
    "roberta", 
    "allenai/biomed_roberta_base", 
    num_labels= 2194,
    args={"num_train_epochs": 100, "train_batch_size": 20, "fp16_opt_level" : 'O0',
     "overwrite_output_dir": True,  "learning_rate": 4e-5,
          "max_seq_length": 128, "save_eval_checkpoints": False, "save_model_every_epoch": False,
         "output_dir": "outputs/",
    "cache_dir": "cache/"}
)

roberta.train_model(temp)

KeyboardInterrupt: 

In [15]:
predictions, raw_outputs = roberta.predict(test['text'])
del test['labels']
del test['probabilities']
test['labels'] = predictions
test['probabilities'] = raw_outputs.tolist()
test.to_csv('robertaD.csv', header = True, index = True)